In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC #selenium에서 사용할 모듈 import
from selenium.common.exceptions import NoSuchElementException 

import time
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


def element_xpath(xpath):
    try:
        element = driver.find_element(By.XPATH, xpath).text
        return element
    except:
        element = ''
        return element


driver = webdriver.Chrome("/Users/2nan22/Desktop/datacrush/ticketcaptin/chromedriver") #selenium 사용에 필요한 chromedriver.exe 파일 경로 지정

df = pd.read_excel('/Users/2nan22/Desktop/datacrush/ticketcaptin/store_list.xlsx')

### TODO : 1) 네이버 지도 입력 시, 업체 리스트가 뜰 때도 있고 바로 업체 페이지로 갈 때를 구분하는 코드 구현
###        2) 업체 검색 시, 검색 결과가 없을 경우 pass

for i in range(len(df)):

    driver.get("https://map.naver.com/v5/") #네이버 신 지도

    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
        ) #입력창이 뜰 때까지 대기
    except:
        pass
    finally:
        pass

    store_name = df['상호명'][i]
    
    search_box = driver.find_element(By.CLASS_NAME, "input_search")

    time.sleep(2)

    search_box.send_keys(store_name)
    search_box.send_keys(Keys.ENTER) #검색창에 상호명 입력

    time.sleep(7) #화면 표시 기다리기

    #TODO : 가게 세부 페이지인지, 상호 리스트가 뜨는지 document의 특정 값으로 판단할 수 있는지 확인
    if 'place' not in driver.current_url:
        
        search_frame = driver.find_element(By.CSS_SELECTOR, "iframe#searchIframe")
        driver.switch_to.frame(search_frame)

        try:
            element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div/div/div[1]/ul/li[1]/div[1]/a/div/div/span[1]')))
            driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[1]/ul/li[1]/div[1]/a/div/div/span[1]').click()
        except:
            driver.find_element(By.XPATH, '/html/body/div[3]/div/div[2]/div[1]/ul/li[1]/div[2]/a[1]/div/div/span[1]').click()
            pass
        finally:
            pass
        
        time.sleep(5)

        driver.switch_to.default_content()

        time.sleep(5)

        try:
            entry_frame = driver.find_element(By.CSS_SELECTOR, "iframe#entryIframe")
            driver.switch_to.frame(entry_frame)
        except:
            pass
    else:
        entry_frame = driver.find_element(By.CSS_SELECTOR, "iframe#entryIframe")
        driver.switch_to.frame(entry_frame)


    time.sleep(5)

    # 변수 초기화
    rate = ''
    address = ''
    menu = ''
    link = ''
    visitor_review_cnt = ''
    blog_review_cnt = ''
    naver_name = ''
    naver_category = ''
        

    # 평점
    rate = element_xpath("/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]/em")
    # 주소
    try:
        address = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div/div[6]/div/div[2]/div/ul/li[1]/div/a/span[1]").text
    except NoSuchElementException:
        address = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div/div[6]/div/div[1]/div/ul/li[1]/div/a/span[1]").text
    except:
        address = ''
    # 메뉴
    try:
        menu = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div/div[6]/div/div[3]/div[1]/ul").text
    except NoSuchElementException:
        menu = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div/div[6]/div/div[2]/div/ul").text
    except:
        menu = ''
    # 방문자 리뷰 수
    visitor_review_cnt = element_xpath("/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[2]/a/em")
    # 블로그 리뷰 수
    blog_review_cnt = element_xpath("/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[3]/a/em")
    # 네이버 상호명
    naver_name = element_xpath("/html/body/div[3]/div/div/div/div[2]/div[1]/div[1]/span[1]")
    # 네이버 카테고리
    naver_category = element_xpath("/html/body/div[3]/div/div/div/div[2]/div[1]/div[1]/span[2]")
    # 링크
    link = driver.current_url


    df['네이버 평점'].loc[i] = rate
    df['위치'].loc[i] = address
    df['링크'].loc[i] = link
    df['방문자 리뷰 수'].loc[i] = visitor_review_cnt
    df['블로그 리뷰 수'].loc[i] = blog_review_cnt
    df['네이버 상호명'].loc[i] = naver_name
    df['네이버 카테고리'].loc[i] = naver_category

    
    # 그림 같이 메뉴판 mpoxR 메뉴하나이름가격 MN48z 메뉴명 MENyI 가격 gl2cc
    # 그냥 메뉴판 jnwQZ 메뉴하나이름가격 gHmZ_ 메뉴명 place_bluelink ihmWt 가격 awlpp
    test = driver.find_element(By.CLASS_NAME, 'MENyI').text
    aa = driver.find_element(By.CLASS_NAME, 'gl2cc').text

    print(test)
    print(aa)

    if menu is not '':
        menu_list = menu.split('\n')
        for menu in menu_list:
            if menu == '대표':
                menu_list.remove(menu)
            elif menu == '사진':
                menu_list.remove(menu)

        menu_final = []
        menu_price = {}

        for ml in range(int(len(menu_list) / 2)):
            menu_price[f'메뉴_{ml}'] = {'음식명' : menu_list[2*ml], '가격' : menu_list[2*ml+1]}
        menu_final.append(menu_price)
        df['대표메뉴'].loc[i] = menu_final

    else:
        menu_final = menu
        df['대표메뉴'].loc[i] = menu_final

    print(store_name)

    time.sleep(5)

df.head()
df.to_excel('./store_list_ver_0.2.xlsx', index=False, encoding='utf-8')




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".MENyI"}
  (Session info: chrome=105.0.5195.125)
Stacktrace:
0   chromedriver                        0x000000010994e788 chromedriver + 4515720
1   chromedriver                        0x00000001098d29d3 chromedriver + 4008403
2   chromedriver                        0x000000010956512a chromedriver + 413994
3   chromedriver                        0x000000010959c873 chromedriver + 641139
4   chromedriver                        0x000000010959ca61 chromedriver + 641633
5   chromedriver                        0x00000001095cef64 chromedriver + 847716
6   chromedriver                        0x00000001095b97fd chromedriver + 759805
7   chromedriver                        0x00000001095ccbd9 chromedriver + 838617
8   chromedriver                        0x00000001095b9603 chromedriver + 759299
9   chromedriver                        0x000000010958f990 chromedriver + 588176
10  chromedriv

In [63]:
!pip install haversine

In [30]:
import pandas as pd
from geopy.geocoders import Nominatim
from haversine import haversine

excel_path = './store_list_ver_0.2.xlsx'
latlng_company = (37.5577182, 126.9253904)

def geocoding(address):
    """
    주소 기반 위도 경도 얻기
    """
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    try:
        geo = geolocoder.geocode(address)
    except:
        geo = geolocoder.geocode(' '.join(address.split(' ')[:4]))
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd

def get_lat_lng_distance(excel_path):
    """
    도로명 주소 기반 위도 경도 산출 후
    회사와의 거리 산출
    """
    df = pd.read_excel(excel_path)
    df['위도_경도'] = df['위치'].apply(lambda x: geocoding(x))
    df['회사와의 거리'] = df['위도_경도'].apply(lambda x: round(haversine((float(x['lat']),float(x['lng'])), latlng_company, unit='m'), 2))

    return df

df = get_lat_lng_distance(excel_path)
df.to_excel('./store_list_ver_0.3.xlsx', index=False, encoding='utf-8')

In [34]:
round(haversine((37.5581051, 126.9260261), latlng_company, unit='m'), 2)

70.65

In [36]:
def obj():
    rate = ''
    address = ''
    menu = ''
    link = ''
    visitor_review_cnt = ''
    blog_review_cnt = ''
    naver_name = ''
    naver_category = ''

    return rate, address, menu, link, visitor_review_cnt, blog_review_cnt, naver_name, naver_category


rate = '2'
rate, address, menu, link, visitor_review_cnt, blog_review_cnt, naver_name, naver_category = obj()
rate

''